# Unlocking Investment Insights Without Moving Your Data: Lakehouse Federation Meets FactSet

**How to combine sensitive on-premise portfolio data with third-party financial intelligence - without compromising security or compliance**

---

## The Challenge: Your Data Lives Everywhere

In today's enterprise landscape, data rarely sits in one place. Financial institutions, asset managers, and individual investors face a common challenge: their most sensitive data—portfolio holdings, trading positions, client information—often resides in on-premise databases behind corporate firewalls. Meanwhile, the third-party data they need for analysis—market data, analyst estimates, financial fundamentals—increasingly lives in the cloud.

Traditional approaches force an uncomfortable choice:

1. **Move everything to the cloud** — But compliance, security policies, or data residency requirements may prohibit this
2. **Keep everything on-premise** — But you miss out on the rich ecosystem of cloud-native data and analytics
3. **Build complex ETL pipelines** — But this creates data duplication, staleness issues, and maintenance overhead

What if you could query your on-premise data *as if* it were in the cloud, joining it seamlessly with marketplace data—without ever moving a single row?

Enter **Databricks Lakehouse Federation**.

---

## The Solution: Query Anywhere, Move Nothing

Lakehouse Federation enables you to create **federated connections** to external databases—SQL Server, PostgreSQL, MySQL, Snowflake, and more—and query them directly from Databricks using standard SQL. The data stays exactly where it is. No copying. No syncing. No compliance headaches.

This opens up a powerful pattern for hybrid analytics:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                        Databricks Lakehouse                             │
│  ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐     │
│  │   FactSet Data  │    │  Federated View │    │   AI/BI         │     │
│  │  (Marketplace)  │───▶│   (JOIN layer)  │───▶│   Dashboard     │     │
│  └─────────────────┘    └────────┬────────┘    └─────────────────┘     │
│                                  │                                      │
└──────────────────────────────────┼──────────────────────────────────────┘
                                   │ Federated Query
                                   │ (data never moves)
                                   ▼
                    ┌─────────────────────────────┐
                    │   On-Premise SQL Server     │
                    │   ┌─────────────────────┐   │
                    │   │  Portfolio Holdings │   │
                    │   │  (first-party data) │   │
                    │   └─────────────────────┘   │
                    └─────────────────────────────┘
```

---

## A Real-World Example: Personal Portfolio Analysis

Let's walk through a concrete example. Imagine you're an investor (or a wealth management firm) with:

- **First-party data**: Your portfolio holdings—which stocks you own and how many shares—stored in an on-premise SQL Server database
- **Third-party data**: FactSet financial data from the Databricks Marketplace, including earnings fundamentals and analyst estimates

Your goal: Build an investment dashboard that shows your projected earnings, identifies high-risk positions, and recommends actions—all without exposing your holdings data to the cloud.

### Step 1: Set Up the Federated Connection

First, create a connection to your on-premise database. In Databricks, this is done through Unity Catalog:

```sql
-- Create a connection to the on-premise SQL Server
CREATE CONNECTION IF NOT EXISTS my_portfolio_connection
TYPE sqlserver
OPTIONS (
    host '<your-sql-server-host>',
    port '1433',
    user SECRET('my-scope', 'sql-user'),
    password SECRET('my-scope', 'sql-password')
);

-- Create a federated catalog that maps to your on-prem database
CREATE FOREIGN CATALOG IF NOT EXISTS mp_portfolio_federated
USING CONNECTION my_portfolio_connection
OPTIONS (database 'PortfolioDB');
```

That's it. Your on-premise `equity_holdings` table is now queryable as `mp_portfolio_federated.dbo.equity_holdings`—but the data never leaves your SQL Server.

### Step 2: Understand the FactSet Data Model

FactSet data from the Marketplace uses internal security identifiers (`fsym_id`) rather than familiar stock tickers. To join with your portfolio data, you need to traverse their symbology tables:

```
Your Portfolio          FactSet Symbology           FactSet Financials
┌─────────────┐        ┌─────────────────┐         ┌─────────────────┐
│ symbol      │───────▶│ ticker_region   │────────▶│ fsym_id         │
│ shares_held │        │ fsym_id         │         │ ff_eps_basic    │
└─────────────┘        └─────────────────┘         │ ff_net_income   │
   MSFT-US                  MSFT-US                │ ff_sales        │
                           R5S7R2-R               └─────────────────┘
```

The key insight: FactSet uses a `-US` suffix for US-listed securities (e.g., `MSFT-US` for Microsoft).

### Step 3: Build the Federated Query

Now comes the magic—joining your on-premise holdings with FactSet data in a single SQL query:

```sql
CREATE OR REPLACE VIEW mp_catalog.analytics.my_portfolio_dashboard AS
WITH portfolio_with_financials AS (
    SELECT
        p.symbol,
        p.shares_held,
        p.instrument_type,

        -- Current fundamentals from FactSet
        ff.FF_EPS_BASIC as eps_current,
        ff.FF_NET_INCOME as net_income,
        ff.FF_SALES as revenue,
        ff.FF_GROSS_INC as gross_income,

        -- Calculate my earnings based on shares held
        (p.shares_held * ff.FF_EPS_BASIC) as my_current_annual_earnings,

        -- Analyst estimates for forward-looking projections
        fe.FE_MEAN as consensus_eps_estimate,
        (p.shares_held * fe.FE_MEAN) as my_projected_annual_earnings,

        -- Growth metrics
        CASE
            WHEN ff.FF_EPS_BASIC > 0
            THEN ((fe.FE_MEAN - ff.FF_EPS_BASIC) / ff.FF_EPS_BASIC) * 100
        END as projected_eps_growth_pct,

        -- Risk indicators
        CASE
            WHEN ff.FF_DEBT_LT > 0 AND ff.FF_COM_EQ > 0
            THEN ff.FF_DEBT_LT / ff.FF_COM_EQ
        END as debt_to_equity_ratio

    FROM mp_portfolio_federated.dbo.equity_holdings p  -- On-premise data (federated)

    -- Join to FactSet symbology to get the internal ID
    INNER JOIN mp_factset_data.sym_v1.sym_ticker_region str
        ON UPPER(p.symbol) = str.ticker_region

    -- Join to security mapping for fundamentals
    INNER JOIN mp_factset_data.ff_v3.ff_sec_map fsm
        ON str.fsym_id = fsm.fsym_id

    -- Get the latest annual financials
    INNER JOIN mp_factset_data.ff_v3.ff_basic_af ff
        ON fsm.ff_co_af_id = ff.ff_co_af_id
        AND ff.ff_fp_ind = 'A'  -- Annual figures

    -- Get analyst consensus estimates
    LEFT JOIN mp_factset_data.fe_v4.fe_basic_af fe
        ON fsm.fsym_id = fe.fsym_id
        AND fe.fe_fp_ind = 'A'
)
SELECT
    *,
    -- Risk assessment
    CASE
        WHEN debt_to_equity_ratio > 2.0 THEN 'High Risk'
        WHEN debt_to_equity_ratio > 1.0 THEN 'Medium Risk'
        ELSE 'Low Risk'
    END as risk_assessment,

    -- Investment recommendation
    CASE
        WHEN projected_eps_growth_pct > 15 AND debt_to_equity_ratio < 1.0 THEN 'Strong Buy'
        WHEN projected_eps_growth_pct > 10 THEN 'Buy More'
        WHEN projected_eps_growth_pct < -10 THEN 'Consider Selling'
        WHEN debt_to_equity_ratio > 2.0 THEN 'Review - High Debt'
        ELSE 'Hold'
    END as action_recommendation

FROM portfolio_with_financials;
```

**What just happened?**
- We queried the on-premise `equity_holdings` table through federation
- We joined it with FactSet symbology, fundamentals, and estimates
- We calculated personalized metrics (my projected earnings based on my shares)
- We generated risk assessments and recommendations

All in a single query. The portfolio data never left the on-premise SQL Server.

### Step 4: Build the Dashboard

With the federated views in place, creating an AI/BI dashboard is straightforward:

![Portfolio Dashboard](dashboard_screenshot.png)

The dashboard includes:
- **KPI Counters**: Total holdings, current earnings, projected earnings
- **Risk Distribution**: Pie chart showing High/Medium/Low risk positions
- **Investment Recommendations**: Distribution of Buy/Hold/Sell signals
- **Top Performers**: Bar chart comparing current vs. projected earnings
- **Action Items**: Table of stocks requiring immediate attention

---

## The Technical Architecture

Here's what's happening under the hood:

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         Databricks Workspace                                │
│                                                                             │
│  ┌──────────────────┐                                                       │
│  │ Unity Catalog    │                                                       │
│  │ ┌──────────────┐ │    ┌─────────────────────────────────────────────┐   │
│  │ │mp_portfolio_ │ │    │              SQL Warehouse                  │   │
│  │ │federated     │◀┼────│  Executes federated queries, pushes down    │   │
│  │ │(foreign)     │ │    │  predicates to source for optimal perf      │   │
│  │ └──────────────┘ │    └─────────────────────────────────────────────┘   │
│  │ ┌──────────────┐ │                        │                              │
│  │ │mp_factset_   │ │                        │                              │
│  │ │data          │ │                        ▼                              │
│  │ │(delta share) │ │    ┌─────────────────────────────────────────────┐   │
│  │ └──────────────┘ │    │              AI/BI Dashboard                │   │
│  │ ┌──────────────┐ │    │  Visualizes combined first-party and       │   │
│  │ │mp_catalog    │ │    │  third-party data in real-time             │   │
│  │ │.analytics    │ │    └─────────────────────────────────────────────┘   │
│  │ │(views)       │ │                                                       │
│  │ └──────────────┘ │                                                       │
│  └──────────────────┘                                                       │
└───────────────────────────────────────────────────────────────────────────┬─┘
                                                                            │
                                    Federated Connection                    │
                                    (secure, encrypted)                     │
                                                                            │
┌───────────────────────────────────────────────────────────────────────────▼─┐
│                        On-Premise SQL Server                                │
│                                                                             │
│  ┌─────────────────────────────────────────────────────────────────────┐   │
│  │  PortfolioDB.dbo.equity_holdings                                    │   │
│  │  ┌──────────┬─────────────┬─────────────────┐                       │   │
│  │  │ symbol   │ shares_held │ instrument_type │                       │   │
│  │  ├──────────┼─────────────┼─────────────────┤                       │   │
│  │  │ MSFT-US  │ 150         │ Stock           │                       │   │
│  │  │ AAPL-US  │ 200         │ Stock           │                       │   │
│  │  │ GOOGL-US │ 50          │ Stock           │                       │   │
│  │  └──────────┴─────────────┴─────────────────┘                       │   │
│  └─────────────────────────────────────────────────────────────────────┘   │
│                                                                             │
│  Data NEVER leaves this server - only query results are returned           │
└─────────────────────────────────────────────────────────────────────────────┘
```

### Key Benefits

| Traditional Approach | With Lakehouse Federation |
|---------------------|---------------------------|
| Copy data to cloud (compliance risk) | Data stays on-premise |
| Build and maintain ETL pipelines | Zero data movement |
| Stale data (batch updates) | Real-time query results |
| Data duplication costs | No storage overhead |
| Complex security model | Existing security preserved |

---

## Getting Started

### Prerequisites

1. **Databricks Workspace** with Unity Catalog enabled
2. **FactSet Data** from the Databricks Marketplace (subscribe to FactSet Fundamentals and Estimates)
3. **On-premise database** accessible via JDBC (SQL Server, PostgreSQL, MySQL, etc.)
4. **Network connectivity** between Databricks and your on-premise database (via VPN, Private Link, or public endpoint with IP allowlisting)

### Quick Start

```sql
-- 1. Subscribe to FactSet data in the Marketplace
--    (This creates a shared catalog in your workspace)

-- 2. Create connection to your on-premise database
CREATE CONNECTION portfolio_conn TYPE sqlserver
OPTIONS (host 'your-server.database.windows.net', port '1433',
         user SECRET('scope', 'user'), password SECRET('scope', 'pwd'));

-- 3. Create federated catalog
CREATE FOREIGN CATALOG portfolio_federated
USING CONNECTION portfolio_conn OPTIONS (database 'YourDB');

-- 4. Query across both sources
SELECT p.symbol, p.shares_held, ff.FF_EPS_BASIC
FROM portfolio_federated.dbo.holdings p
JOIN factset_catalog.ff_v3.ff_basic_af ff ON ...
```

---

## Beyond Portfolio Analysis

While we used investment portfolios as our example, this pattern applies broadly:

- **Healthcare**: Join on-premise patient records (HIPAA-protected) with cloud-based drug interaction databases
- **Retail**: Combine in-store POS data with cloud-based customer behavior analytics
- **Manufacturing**: Link factory floor sensor data with cloud-based supply chain intelligence
- **Financial Services**: Merge core banking data with real-time market feeds

The common thread: **sensitive first-party data stays protected** while still enabling **rich analytics with third-party data**.

---

## Conclusion

Lakehouse Federation fundamentally changes the economics of hybrid data analytics. Instead of the expensive, risky, and time-consuming process of moving data to the cloud, you can now bring the compute to the data.

In our example, we built a complete investment portfolio dashboard that:
- Queries on-premise holdings data in real-time
- Enriches it with FactSet financial intelligence
- Generates risk assessments and recommendations
- Visualizes everything in an interactive dashboard

All without copying a single row of portfolio data to the cloud.

**The future of data analytics isn't about moving all your data to one place—it's about querying it wherever it lives.**

---

### Resources

- [Databricks Lakehouse Federation Documentation](https://docs.databricks.com/en/query-federation/index.html)
- [FactSet on Databricks Marketplace](https://marketplace.databricks.com/search?query=factset)
- [Unity Catalog Foreign Tables](https://docs.databricks.com/en/query-federation/foreign-tables.html)
- [Sample Code Repository](https://github.com/databricks/lakehouse-federation-examples)

---

*Ready to try it yourself? Sign up for a [Databricks free trial](https://databricks.com/try-databricks) and explore the Marketplace to get started with federated analytics today.*
